In [4]:
import pandas as pd

input_parameters = 14
layer_size = 5
layer_count = 3
classification_size = 3
learning_rate = 0.001

In [5]:
from sampo.scheduler.selection.neural_net import NeuralNet, load_dataset

model = NeuralNet(input_parameters, layer_size, layer_count, classification_size)
list(model.parameters())

[Parameter containing:
 tensor([[-1.5927e-01, -1.9320e-01,  9.5098e-02, -2.0807e-01,  1.7157e-01,
          -1.6885e-02, -6.7384e-02, -2.5003e-01, -2.0265e-01, -6.5255e-03,
          -1.0617e-01, -1.9848e-01,  6.7585e-02],
         [-4.9080e-02,  1.8151e-01,  1.3348e-01,  1.4507e-01,  2.3773e-01,
           8.8335e-02,  3.3662e-02,  4.5203e-02, -1.3676e-01, -4.6430e-02,
          -2.6078e-01,  1.8373e-01,  1.0592e-01],
         [-1.5682e-01,  3.6403e-02,  6.2144e-02,  9.2655e-05,  3.5083e-02,
          -2.3186e-01, -1.0585e-01, -2.0260e-01,  2.7534e-01,  2.5723e-01,
          -2.0047e-01,  2.3451e-02, -2.0298e-01],
         [ 8.9845e-02,  6.6567e-02, -2.3717e-01,  1.9971e-01,  3.2292e-02,
          -1.0897e-01,  1.3988e-01,  2.1391e-01, -2.3284e-01,  9.0283e-02,
           1.8073e-01,  2.4850e-01,  1.6076e-01],
         [ 7.0289e-02,  1.5040e-01,  3.4290e-02,  2.4460e-01, -2.1923e-02,
           1.1609e-01, -1.0063e-01, -3.9139e-02, -8.3488e-02, -1.6468e-01,
          -1.5083e-01,  2.4

In [6]:
# for i in range(10):
#     print(f'Loop {i}/10')
x_train, x_test, y_train, y_test = load_dataset('dataset.csv')
model.fit(x_train, y_train, 100)
list(model.parameters())

[Parameter containing:
 tensor([[-1.5927e-01, -1.9320e-01,  9.5098e-02, -2.0807e-01,  1.7157e-01,
          -1.6885e-02, -6.7384e-02, -2.5003e-01, -2.0265e-01, -6.5255e-03,
          -1.0617e-01, -1.9848e-01,  6.7585e-02],
         [-4.9080e-02,  1.8151e-01,  1.3348e-01,  1.4507e-01,  2.3773e-01,
           8.8335e-02,  3.3662e-02,  4.5203e-02, -1.3676e-01, -4.6430e-02,
          -2.6078e-01,  1.8373e-01,  1.0592e-01],
         [-1.5682e-01,  3.6403e-02,  6.2144e-02,  9.2655e-05,  3.5083e-02,
          -2.3186e-01, -1.0585e-01, -2.0260e-01,  2.7534e-01,  2.5723e-01,
          -2.0047e-01,  2.3451e-02, -2.0298e-01],
         [ 8.9845e-02,  6.6567e-02, -2.3717e-01,  1.9971e-01,  3.2292e-02,
          -1.0897e-01,  1.3988e-01,  2.1391e-01, -2.3284e-01,  9.0283e-02,
           1.8073e-01,  2.4850e-01,  1.6076e-01],
         [ 7.0289e-02,  1.5040e-01,  3.4290e-02,  2.4460e-01, -2.1923e-02,
           1.1609e-01, -1.0063e-01, -3.9139e-02, -8.3488e-02, -1.6468e-01,
          -1.5083e-01,  2.4

In [7]:
y_test

[tensor([0., 0., 1.]), tensor([0., 1., 0.]), tensor([1., 0., 0.])]

In [10]:
model.score(x_test, y_test)

In [15]:
import pandas as pd
from sampo.scheduler.selection.neural_net import NeuralNet

train_dataset = pd.read_csv('dataset.csv', index_col='index')
for col in train_dataset.columns[:-1]:
    train_dataset[col] = train_dataset[col].apply(lambda x: float(x))

In [16]:
input_parameters = 14
layer_size = 5
layer_count = 3
classification_size = 3
learning_rate = 0.0001

model = NeuralNet(input_parameters, layer_size, layer_count, classification_size, learning_rate)

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
index,,,,,,,,,,,,,,
0,127.0,0.859823,46.0,29.814961,46.0,1.0,1.0,3.0,93.0,424.0,729.0,881.0,258.0,0
2,190.0,0.906255,68.0,30.084211,68.0,1.0,1.0,4.0,138.0,527.0,892.0,976.0,292.0,0
1,112.0,1.121457,34.0,35.151786,34.0,1.0,1.0,3.0,69.0,412.0,667.0,883.0,226.0,0
3,182.0,0.977690,66.0,33.574176,66.0,1.0,1.0,3.0,66.0,356.0,1260.0,1344.0,900.0,1
5,156.0,0.719637,58.0,28.128205,58.0,1.0,1.0,3.0,94.0,400.0,800.0,860.0,356.0,1
4,127.0,1.049286,41.0,32.751969,41.0,1.0,1.0,3.0,71.0,482.0,749.0,1009.0,242.0,1
6,191.0,0.644718,66.0,27.039267,66.0,1.0,1.0,3.0,90.0,646.0,1248.0,1568.0,580.0,2
7,133.0,0.757486,38.0,29.436090,38.0,1.0,1.0,3.0,77.0,560.0,865.0,1193.0,274.0,2
8,195.0,0.614343,78.0,27.212821,78.0,1.0,1.0,3.0,78.0,408.0,1228.0,1304.0,804.0,2


In [17]:
train_dataset

Test set: Average loss: 0.5171, Accuracy: 0/3 (0.00%)

Test set: Average loss: 0.5171, Accuracy: 2/3 (66.67%)

Test set: Average loss: 0.5171, Accuracy: 0/3 (0.00%)



[0.0, 66.66666666666667, 0.0]

In [4]:
from sampo.scheduler.selection.neural_net import one_hot_encode, cross_val_score
from sklearn.model_selection import KFold
import torch
import torch.nn as nn

s = cross_val_score(train_dataset=train_dataset,
                      target_column='label',
                      model=model,
                      epochs=100,
                      folds=10,
                      shuffle=True,
                      random_state=42)

Epoch [1/100], Step [100/135], Loss: 0.5514
Epoch [2/100], Step [100/135], Loss: 0.5514
Epoch [3/100], Step [100/135], Loss: 0.5514
Epoch [4/100], Step [100/135], Loss: 0.5514
Epoch [5/100], Step [100/135], Loss: 0.5514
Epoch [6/100], Step [100/135], Loss: 0.5514
Epoch [7/100], Step [100/135], Loss: 0.5514
Epoch [8/100], Step [100/135], Loss: 0.5514
Epoch [9/100], Step [100/135], Loss: 0.5514
Epoch [10/100], Step [100/135], Loss: 0.5514
Epoch [11/100], Step [100/135], Loss: 0.5514
Epoch [12/100], Step [100/135], Loss: 0.5514
Epoch [13/100], Step [100/135], Loss: 0.5514
Epoch [14/100], Step [100/135], Loss: 0.5514
Epoch [15/100], Step [100/135], Loss: 0.5514
Epoch [16/100], Step [100/135], Loss: 0.5514
Epoch [17/100], Step [100/135], Loss: 0.5514
Epoch [18/100], Step [100/135], Loss: 0.5514
Epoch [19/100], Step [100/135], Loss: 0.5514
Epoch [20/100], Step [100/135], Loss: 0.5514
Epoch [21/100], Step [100/135], Loss: 0.5514
Epoch [22/100], Step [100/135], Loss: 0.5514
Epoch [23/100], Ste

In [5]:
print(s)

[33.333333333333336, 40.0, 33.333333333333336, 40.0, 40.0, 46.666666666666664, 20.0, 60.0, 33.333333333333336, 40.0]
